In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import json
import base64
import requests
from requests.structures import CaseInsensitiveDict
import os

In [ ]:
#utils
def extract_base64_contents(file):
    with open(file, 'rb') as fp:
        image = base64.b64encode(fp.read()).decode('ascii')
    return image

In [ ]:
#consts
dataset_path = "/content/gdrive/MyDrive/MIT_dataset/images/"
dataset_name = "MIT_dataset"

file_extension = ".jpg"
face_id_map = dict() #will contain photo_name : face_id
result_map = dict() 

url_base = "https://www.betafaceapi.com/api"
url_upload_photo = url_base + "/v2/media" 
url_create_person = url_base + "/v2/person" 
url_recognize = url_base + "/v2/recognize" 

api_key = 'd45fd466-51e2-4701-8da8-04351c872236'
api_secret = '171e8465-f548-401d-b63b-caf0dc28df5f'

headers = CaseInsensitiveDict()
headers["accept"] = "application/json"
headers["Content-Type"] = "application/json"
data = { 'api_key': api_key, 'api_secret': api_secret,}

In [ ]:
def upload_photo(photo_name):
  global data

  complete_file_path = dataset_path + photo_name + file_extension
  payload = { 
      "file_base64": extract_base64_contents(complete_file_path), 
      "original_filename": photo_name,
      "set_person_id" : photo_name + "@" + dataset_name,
  }

  payload.update(data)

  response = requests.post(url_upload_photo, headers=headers, data=str(payload))
  response_json = response.json()
  
 # print("upload_photo: ", json.dumps(response_json, indent = 4))
       
  if response.status_code != 200 or 'Error' in response_json or not response_json["media"]["faces"]:
    print("UPLOAD PHOTO FAIL: ",  json.dumps(response_json, indent = 4))
    return None
  else:
    return response_json["media"]["faces"][0]["face_uuid"]    

In [ ]:
def init():
  global face_id_map, result_map
  face_id_map.clear()
  result_map.clear()
  
  for filename in sorted(os.listdir(dataset_path)):
    #print(filename)
    
    key = filename.split('.')[0] #get file names without extension
    result_map.update({key : None}) #insert keys in map

    print("Get face_id for ", key, "...")
    face_id = upload_photo(key)

    #face_id person in map
    face_id_map.update({key: face_id})

  print("face_id_map: ", json.dumps(face_id_map, indent = 4))
  #print(json.dumps(result_map, indent = 4))

In [ ]:
def recognize(face_id):
  global data

  if not face_id:
    return dict()

  payload = {  
      "faces_uuids": [ 
        face_id 
      ],
      "targets" : [ 
        "all@" + dataset_name
      ],
      "parameters": "min_match_score:0.0,min_score:0.0"
  }

  payload.update(data)

  response = requests.post(url_recognize, headers=headers, data=str(payload))
  response_json = response.json()
  
  #print("recognize: ", json.dumps(response_json, indent = 4))
       
  if response.status_code != 200 or 'Errors' in response_json:
    print("recognize FAIL: ", json.dumps(response_json, indent = 4)) 
     
  else:

    recognize_result_dict = dict()
    for element in response_json["results"][0]["matches"]:

      extracted_confidence = round(float(element["confidence"]) * 100,2) # 0.86944 -> 86.94
      extracted_person_name = element["person_id"].split('@')[0]

      recognize_result_dict.update({extracted_person_name : extracted_confidence})

    #print(json.dumps(recognize_result_dict, indent = 4))
    return recognize_result_dict

In [ ]:
def add_results():
  global face_id_map, result_map

  for key in result_map.keys():
    
    print("adding results for key:", key, "...")
    recognize_result_dict = recognize(face_id_map.get(key))
    

    #adding missing faces
    for missing_key in result_map.keys():
      if not recognize_result_dict or missing_key not in recognize_result_dict.keys():
        recognize_result_dict.update({missing_key : None})

    result_map.update({key : recognize_result_dict})
  #print(json.dumps(result_map, indent = 4))

In [ ]:
def show_results():
  print("========================================FINAL RESULTS========================================")
  for key, value in sorted (result_map.items()):
    print(key, ":{", end="")
    if value:
      for key2, value2 in sorted (value.items()):
        print(value2, end=", ", flush=True)
      print("}\n")
    else:
      print("None}\n")


In [ ]:
init()
add_results()
show_results()

========================================FINAL RESULTS========================================
0_1 :{100.0, 93.8, None, 93.8, 71.8, None, None, 65.5, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 65.7, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 70.4, 70.4, 67.8, None, 71.4, 70.2, None, 65.4, 65.4, 66.6, None, 66.3, None, }

0_2 :{93.8, 100.0, None, 92.7, 69.7, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 67.6, 66.2, 67.5, None, 69.5, 67.4, None, None, None, None, None, 69.0, 69.4, 68.2, None, 70.5, 68.6, None, None, None, None, None, 66.5, None, }

0_3 :{None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None